In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
import os
from langsmith import traceable
import json
import asyncio
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
AZURE_OPENAI_API_KEY=os.environ["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT=os.environ["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_API_VERSION=os.environ["AZURE_OPENAI_API_VERSION"]

In [7]:
kernel = Kernel()

service = AzureChatCompletion(
    deployment_name="gpt-4o-mini",
    endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

kernel.add_service(service)

In [8]:
research_prompt = """
You are a research agent.

Provide factual explanation and key points for:
{{$input}}
"""

request_settings_researcher = AzureChatPromptExecutionSettings(temperature=0.1,max_tokens=1000)
research_fn = kernel.add_function(plugin_name="team", function_name="researcher", 
                                  prompt=research_prompt, 
                                  prompt_execution_settings=request_settings_researcher)

@traceable(name="Research Agent")
async def run_research(q):
    return await kernel.invoke(research_fn, input=q)

In [9]:
analysis_prompt = """
You are an analyst agent.

Critically evaluate the topic:
{{$input}}

Highlight strengths, weaknesses, risks.
"""
request_settings_analyzer = AzureChatPromptExecutionSettings(temperature=0.1,max_tokens=1000)
analysis_fn = kernel.add_function(plugin_name="team",
                                  function_name="analyst",
                                  prompt=analysis_prompt,
                                  prompt_execution_settings=request_settings_analyzer)

@traceable(name="Analyst Agent")
async def run_analysis(q):
    return await kernel.invoke(analysis_fn, input=q)

In [10]:
synth_prompt = """
You are a synthesis agent.

Combine research and analysis into a balanced final answer.

Research:
{{$research}}

Analysis:
{{$analysis}}
"""
request_settings_synthesizer = AzureChatPromptExecutionSettings(temperature=0.1,max_tokens=1000)
synth_fn = kernel.add_function(plugin_name="team", 
                               function_name="synthesizer", 
                               prompt=synth_prompt,
                               prompt_execution_settings=request_settings_synthesizer)


@traceable(name="Synthesis Agent")
async def run_synth(research, analysis):
    return await kernel.invoke(
        synth_fn,
        research=research,
        analysis=analysis
    )

In [11]:
@traceable(name="Collaboration Stage")
async def collaboration_stage(q):

    research_task = asyncio.create_task(run_research(q))
    analysis_task = asyncio.create_task(run_analysis(q))

    research_res, analysis_res = await asyncio.gather(
        research_task, analysis_task
    )

    return str(research_res), str(analysis_res)

In [12]:
@traceable(name="Multi-Agent Collaboration")
async def collaborative_agent(user_question):

    research, analysis = await collaboration_stage(user_question)

    final_answer = await run_synth(research, analysis)

    return final_answer


In [13]:
answer = await collaborative_agent(
    "How does hybrid retrieval improve RAG systems?"
)

In [ ]:
print(answer)

Hybrid retrieval systems represent a significant advancement in the field of Retrieval-Augmented Generation (RAG) by integrating various retrieval methods to optimize both information retrieval and generation tasks. This synthesis of research and analysis highlights the strengths, weaknesses, and risks associated with hybrid retrieval, providing a comprehensive understanding of its impact on RAG systems.

### Strengths of Hybrid Retrieval in RAG Systems

1. **Enhanced Relevance and Precision**:
   - By combining traditional keyword-based retrieval methods (like TF-IDF or BM25) with neural retrieval techniques (such as dense vector embeddings), hybrid systems can leverage the strengths of both approaches. Traditional methods ensure precision through exact keyword matches, while neural methods enhance semantic understanding, leading to more contextually relevant results.

2. **Improved Coverage and Recall**:
   - Hybrid retrieval systems can access a broader range of data sources, includ